In [1]:
import matplotlib.patches as patches
from scipy.interpolate import interp1d #spline
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

import ibex
ssh = ibex.connect("158.251.88.197")


Cargando ibex...


login: iaraya
pass: ········


In [2]:
class point:
    def __init__(self, x_,y_):
        self.x = x_
        self.y = y_
        
    #SEPARAR X DE Y PARA GRAFICAR (?)
    def separate(points):
        x = []; y = []
        for i in range(len(points)):
            aux = points[i]
            x.append(points[i].x)
            y.append(points[i].y)
        return x,y
        #self.__class__.separate = separate 
        
    def sort2(points):
        points_sorted = []
        (x,y) = point.separate(points)
        while True:
            #Encontrar valor minimo
            #Encontrar su ubicacion el la lista
            #Obtenerlo y añadirlo a points_sorted
            #Eliminarlo de x, y y points
            min_ = min(x)
            idx_min = x.index(min_)
            points_sorted.append(points[idx_min])
            x.remove(min_)
            y.remove(y[idx_min])
            points.remove(points[idx_min])
            if(len(points) == 0):
                break
        return points_sorted
        #self.__class__.sort2 = sort2 

Search Efficient + Spline (Example)
====

**Conexión por SSH**

In [3]:
#ibex.get_instances()

**Interpolación usando spline y search_efficient**

In [3]:
def interpolacion(ssh):
 
    
    instance = "mop-7.txt"
    npoints = 4
    prec_se = 1e-1 #precisión para search_efficient
    prec = 2 #precisión para el solver

    # We obtain the initial box
    b1 = ibex.search_efficient(instance, [-1e8,1e8,-1e8,1e8], mode="minf1", prec=prec_se)[1]
    b2 = ibex.search_efficient(instance, [-1e8,1e8,-1e8,1e8], mode="minf2", prec=prec_se)[1]
    y1 = [b1[0], b2[0]]
    y2 = [b2[1], b1[1]]
    print("inital box:", y1,y2)

    points = []
    x = np.linspace(y1[0], y1[1], num=npoints, endpoint=True)
    y = np.zeros(npoints)
    xpoints = []
    i = 0
    for a in x:
        #print("a : ",a)
        xpoint, ypoint, time = ibex.search_efficient(instance, [a-10*prec_se,a+10*prec_se,y2[0],y2[1]], mode="minf2", prec=prec_se)
        xpoints.append(xpoint)
        #x[i] = ypoint[0]; y[i] = ypoint[1] #ypoint[0] = f1 ypoint[1] = f2
        point_ = point(ypoint[0], ypoint[1])
        points.append(point_)
        i+=1
    #print("xpoints : ", xpoints)
    x,y=point.separate(points)
    
    f = interp1d(x, y) #linear
    f2 = interp1d(x, y, kind='cubic')

    xsol, ysol = ibex.solve(instance, prec=prec)

    xnew = np.linspace(x[0], x[-1], num=41, endpoint=True)

#    plt
#    plt.plot(x, y, 'o', xnew, f(xnew), '-', xnew, f2(xnew), '--', xsol, ysol, '-')
#    plt.legend(['data', 'linear', 'cubic', 'realP'], loc='best')
#    plt.show()
  
    return (points,xnew,xsol,ysol, instance, prec_se,y1,y2,xpoints)

Búsqueda de puntos eficientes
--

**1. Cálculo del punto a interpolar**

In [4]:
def contains(x, evaluated_points):
    for i in range(len(evaluated_points)):
        box = evaluated_points[i]
        #x1,x2,y1,y2
        if (x >= box[0]) and (x <= box[1]) :
            #print("Ya se evaluó")
            return True
    return False

def union(points,evaluated_points):
    x,y=point.separate(points)
    x_aux = x
    y_aux = y
    x_sorted = []
    y_sorted = []
    for i in range(len(evaluated_points)):
        box = evaluated_points[i]
        x_aux.append(box[0])
        x_aux.append(box[1])
        y_aux.append(box[2])
        y_aux.append(box[3])

    while True:
        #Encontrar valor minimo
        #Encontrar su ubicacion el la lista
        #Obtenerlo y añadirlo a points_sorted
        #Eliminarlo de x, y y points
        min_ = min(x_aux)
        idx_min = x_aux.index(min_)
        x_sorted.append(x_aux[idx_min])
        y_sorted.append(y_aux[idx_min])
        x_aux.pop(idx_min)
        y_aux.pop(idx_min)
        if(len(x_aux) == 0):
            break
    
    return x_sorted,y_sorted

In [9]:
#DETERMINA EL AREA EN DONDE SE BUSCARÁ UN PUNTO
def search_point(evaluated_points,points,xnew,xsol,ysol):
    best_i = 0
    is_solution = True
    x,y=point.separate(points)
    x_aux,y_aux = union(points,evaluated_points)
    max_area=0;
    xmid = np.array(1)
#    print("x= = ",x)
#    print("y ANTES= = ",y)
    #ANTES DE VER LA MAYOR DISTANCIA HAY QUE VER SI YA SE EVALUÓ ESA AREA Y SALTAR SI NO ENCONTRÓ SOLUCIÓN
    for i in range(len(x_aux)-1):
        if (contains(x_aux[i], evaluated_points) == True):
            continue
            
        area = (x_aux[i+1]-x_aux[i])*(y_aux[i]-y_aux[i+1])
#        print("area = ", area)
        if max_area < area:
#            print("max_area = ", max_area, " area = ", area)
            max_area=area
            if(np.any(x == x_aux[i]) == False):
                is_solution = False
                
                #xmid = np.array((x_aux[best_i]+x_aux[best_i+1])/2)
            else:
                best_i = x.index(x_aux[i])
                is_solution = True
            best = i
        xmid = np.array((x_aux[best]+x_aux[best+1])/2) #punto a interpolar
#    print("xmid = ",xmid)
#    print("x DESPUES = ",x)
#    print("y DESPUES = ",y)
    if is_solution == True:
        f2 = interp1d(x, y, kind='cubic')

#    plt.plot(x, y, 'o', xnew, f2(xnew), 'g--', xsol, ysol, 'y-', xmid, f2(xmid), 'rx')
#    plt.legend(['data', 'cubic', 'realP', 'prediction'], loc='best')
#    plt.show()
    
    return (evaluated_points,points, xmid,f2,best_i,is_solution)

**2. Creación de small_box**

In [6]:
def smallBox(prec_se,evaluated_points,points,y1,y2, xmid,f2,xsol,ysol,xpoints,best_i,is_solution):
    x,y=point.separate(points)
    h = 0.10*(y2[1]-y2[0]) #altura de caja
    w = 0.10*(y1[1]-y1[0]) #ancho de caja


    small_box = [xmid-10*prec_se,xmid+10*prec_se,y2[0],y2[1]]#[xmid-w/2, xmid+w/2, f2(xmid)-h/2, f2(xmid)+h/2]
    #print("xmid en smallbox : ",xmid)

    if(is_solution == True):
        box_x = []
        for i in range(len(xpoints[0])):
            box_x.append(np.minimum(xpoints[best_i][i],xpoints[best_i+1][i]))
            box_x.append(np.maximum(xpoints[best_i][i],xpoints[best_i+1][i]))

    # Create figure and axes
#    fig,ax = plt.subplots(1)

    # Create a Rectangle patch
#    rect = patches.Rectangle((xmid-w/2,f2(xmid)-h/2),w,h,linewidth=2,edgecolor='r',facecolor='r')

    # Add the patch to the Axes
#    ax.add_patch(rect)
#    plt.plot(x, y, 'o', xsol, ysol, 'y-')
#    plt.legend(['data', 'realP', 'smallbox'], loc='best'); plt.show()
    
    return (evaluated_points,points,small_box,h,w,box_x,xpoints)

**Buscando en small_box**

In [7]:
def search_small_box(evaluated_points,points,instance,small_box,xsol,ysol,h,w,xpoints,best_i,box_x):
    x,y = point.separate(points)
    aux_x, aux_y, time =  ibex.search_efficient(instance, small_box, mode="efficient", prec=prec_se, box_x=box_x)
    xx, yy = aux_y
    #print("xx en search_small_box : ",xx)
    if xx==np.inf or yy==np.inf:
        #print("small_box : ", small_box)
        #small_box = lista
        #index np.array
        evaluated_points.append(small_box)
    else:
        x_=np.append(x,xx);# x_.sort() #y1
        y_=np.append(y,yy);# y_[::-1].sort() #y2
        xpoints.append(aux_x)

        point_ = point(xx,yy)
        points.append(point_)
        points = point.sort2(points)

#        fig,ax = plt.subplots(1)
#        rect = patches.Rectangle((xmid-w/2,f2(xmid)-h/2),w,h,linewidth=1,edgecolor='y',facecolor='none')
#        ax.add_patch(rect)
#        plt.plot(x_, y_, 'o', xsol, ysol, '-')
#        plt.legend(['data', 'realP'], loc='best')
#        plt.show()

    print("time (search efficient):", time)
    return (evaluated_points, points,xpoints, time)

**4. Actualizar vectores. Luego, volver a 1**

In [10]:
(points,xnew,xsol,ysol,instance,prec_se,y1,y2,xpoints) = interpolacion(ssh)
evaluated_points = []
total_time = 0
add = True
#pasar x e y a un vector de point, llamado points, reemplazará parametros x e y en todo 
j = 0
while True:
    (evaluated_points,points, xmid,f2,best_i, is_solution) = search_point(evaluated_points,points,xnew,xsol,ysol)
    (evaluated_points,points,small_box,h,w,box_x, xpoints) = smallBox(prec_se,evaluated_points,points,y1,y2, xmid,f2,xsol,ysol,xpoints,best_i,is_solution)
    (evaluated_points,points,xpoints,time) = search_small_box(evaluated_points,points,instance,small_box,xsol,ysol,h,w,xpoints,best_i,box_x)
    j+=1
    total_time+=time
    print("iteracion numero : ",j)
    for i in range(len(points)):
        a = points[i]
        print("points : ", a.x, a.y)
    print("-----------------------------------------------------")
    if len(points) == 50:
        break
    
print("tiempo total : ", total_time)

inital box: [0.0, 40.0] [0.0, 40.0]
[0.991170411642913, 14.3277565214336, 27.646429295359, 39.3944001027697]
time (search efficient): 4.38217600000001
iteracion numero :  1
points :  0.991170411642913 19.6283783694685
points :  6.65946346653826 7.08052478477626
points :  14.3277565214336 2.35497306246351
points :  27.646429295359 0.224545853627038
points :  39.3944001027697 0.0678874983186377
-----------------------------------------------------
[0.991170411642913, 6.65946346653826, 14.3277565214336, 27.646429295359, 39.3944001027697]
time (search efficient): 4.36981700000001
iteracion numero :  2
points :  0.991170411642913 19.6283783694685
points :  4.04045281699467 10.5228676116205
points :  6.65946346653826 7.08052478477626
points :  14.3277565214336 2.35497306246351
points :  27.646429295359 0.224545853627038
points :  39.3944001027697 0.0678874983186377
-----------------------------------------------------
[0.991170411642913, 4.04045281699467, 6.65946346653826, 14.3277565214336, 

ValueError: Expect x to be a 1-D sorted array_like.